# **Unveiling ENSO Dynamics:** Insights from Topological Data Analysis


## Dependencias

In [ ]:
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

from umap import UMAP
# General Software Dependencies
# import datetime as dt
from datetime import datetime

# Mathematical Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px


# Dependencias específicas
from gtda.time_series import SingleTakensEmbedding
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import BettiCurve
from gtda.plotting import plot_point_cloud
from sklearn.manifold import SpectralEmbedding
from sklearn.cluster import KMeans
import kmapper as km

# Plotly parameters
# Calculate the desired width and height ratio
width_ratio = 10
height_ratio = 10
# Calculate the desired width based on the height and ratio
desired_height = 450  # Choose an appropriate height value
desired_width = desired_height * width_ratio / height_ratio
margin=dict(l=30, r=30, t=30, b=30)

plt.style.use('tableau-colorblind10')

## Funciones auxiliares

In [ ]:
def parse_space_separated_file_to_dataframe(filename):
    data = []
    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespaces
            if line:  # Skip empty lines
                items = line.split()  # Split line into separate items
                year = items[0]  # First item is the year
                months = items[1:]  # Remaining items are months
                data.append([year] + months)  # Append year and months to data list
    
    df = pd.DataFrame(data, columns=["Year"] + list(range(1, 13)))
    return df


def parse_general_file(file_name):
    data = []
    with open(file_name, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespaces
            if line:  # Skip empty lines
                items = line.split()  # Split line into separate items
                year = items[0]  # First item is the year
                months = items[1:]  # Remaining items are months
                data.append([year] + months)  # Append year and months to data list

    gen_df = pd.DataFrame(data)
    gen_df.columns = gen_df.iloc[0]
    gen_df = gen_df[1:]
    gen_df['date'] = pd.to_datetime(gen_df["YR"].astype(str) + "-" + gen_df["MON"].astype(str), format="%Y-%m")
    gen_df.set_index('date', inplace=True)
    gen_df.columns = ['year', 'month', 'nino1.2', 'anom_nino1.2',
                  'nino3', 'anom_nino3','nino4', 'anom_nino4',
                  'nino3.4', 'anom_nino3.4']
    
    gen_df = gen_df.apply(pd.to_numeric)
    gen_df = gen_df.sort_index()

    return gen_df


def fit_embedder(embedder: SingleTakensEmbedding, y: np.ndarray, verbose: bool=True):
    """Fits a Takens embedder and displays optimal search parameters."""
    
    y_embedded = embedder.fit_transform(y)
    delay = embedder.time_delay_

    if verbose:
        print(f"Shape of embedded time series: {y_embedded.shape}")
        print(
            f"Optimal embedding dimension is {embedder.dimension_} and time delay is {delay:.4f}"
        )

    return y_embedded, delay


def periodicity_analysis(max_embedding_dimension: int, 
                         max_time_delay: int, 
                         stride: int, 
                         y: pd.Series, 
                         var_name: str,
                         cluster: callable, 
                         persistence: callable):
    

    embedder = SingleTakensEmbedding(
        parameters_type="search",
        n_jobs=-1,
        time_delay=max_time_delay,
        dimension=max_embedding_dimension,
        stride=stride,
    )
    y_embedded, delay = fit_embedder(embedder, y)
    print(f"Time delay: {delay}")

    fig = go.Figure(data=go.Scatter(
                        x=y.index.astype(str), 
                        y=y,
                        marker_color='indianred', 
                        text=f"{var_name} idx")
                    )
    fig.update_layout({"title": f'Time series for: {var_name}',
                    "xaxis": {"title":"Date"},
                    "yaxis": {"title":f"{var_name} idx"},
                    "showlegend": False,
                    "width": desired_width*2.11,
                    "height": desired_height})
    
    fig.write_image(f'docs/img/{var_name}_time_series.png', scale=2)
    fig.show()
        
    y_embedded_trans = cluster.fit_transform(y_embedded)
    proj_series_fig = plot_point_cloud(y_embedded_trans[:, :3])

    margin=dict(l=20, r=20, t=30, b=30)
    proj_series_fig.update_layout(
        title=f'Encaje de {var_name}',
        width=desired_width,
        height=desired_height,
        margin=margin
        )
    
    proj_series_fig.write_image(f'docs/img/{var_name}_projection.png', scale=2)
    proj_series_fig.show()

    pers_vals = persistence.fit_transform(y_embedded_trans[None, :, :])
    pers_fig = persistence.plot(pers_vals)
    pers_fig.update_layout(
        title=f'Diagrama de Persistencia de {var_name}',
        height=desired_height
    )
    pers_fig.write_image(f'docs/img/{var_name}_persistence.png', scale=2)
    pers_fig.show()

    bettis = BettiCurve()
    bet_vals = bettis.fit_transform(pers_vals)
    betti_fig = bettis.plot(bet_vals)
    betti_fig.update_layout(
        title=f'Curvas de Betti de {var_name}',
        width=desired_width,
        height=desired_height
        )
    
    betti_fig.write_image(f'docs/img/{var_name}_betti_curve.png', scale=2)
    betti_fig.show()

## Lectura de datos

In [ ]:
df = parse_space_separated_file_to_dataframe('db/nino34.long.anom.data.txt')
df = df.melt(id_vars=["Year"], var_name="Month", value_name='anom_nino3.4')
df["date"] = pd.to_datetime(df["Year"].astype(str) + "-" + df["Month"].astype(str), format="%Y-%m")
df = df[['date', 'anom_nino3.4']]
df = df.set_index('date')
df['anom_nino3.4'] = df['anom_nino3.4'].astype(float)
df = df.sort_index()

gen_df = parse_general_file('db/sstoi.indices.txt')


## Análisis de periodicidad

### Nino 1.2

In [ ]:
var = 'nino1.2'
delta = 15
start = 1980  - 6*delta
dt_start = datetime(1980, 1, 1)
dt_end = datetime(1995, 1, 1)
data = gen_df[var] # [dt_start:dt_end][var]


max_embedding_dimension = 10
max_time_delay = 10
stride = 1
um  = UMAP(random_state=0, n_components=4)
se = SpectralEmbedding(n_components=4, random_state=0)
y = data
var_name = var
persistence = VietorisRipsPersistence(
    homology_dimensions=[0, 1, 2],
    n_jobs=-1
)

periodicity_analysis(max_embedding_dimension=max_embedding_dimension,
                     max_time_delay=max_time_delay,
                     stride=stride,
                     y=y,
                     var_name=var_name,
                     cluster=se,
                     persistence=persistence
                     )

### Anomalías Nino 1.2

In [ ]:
# 1980, 1995
var = 'anom_nino1.2'
dt_start = datetime(2000, 1, 1)
dt_end = datetime(2020, 1, 1)
data = gen_df[var] #[dt_start:dt_end][var]
data = gen_df[dt_start:dt_end][var]

#UMAP
max_embedding_dimension = 10
max_time_delay = 10
stride = 1
um  = UMAP(random_state=0, n_components=4)
y = data
var_name = var
persistence = VietorisRipsPersistence(
    homology_dimensions=[0, 1, 2],
    n_jobs=-1
)

periodicity_analysis(max_embedding_dimension=max_embedding_dimension,
                     max_time_delay=max_time_delay,
                     stride=stride,
                     y=y,
                     var_name=var_name,
                     cluster=um,
                     persistence=persistence)

### Niño 3

In [ ]:
var = 'nino3'
delta = 15
start = 1980  - 6*delta
dt_start = datetime(1992, 1, 1)
dt_end = datetime(2005, 1, 1)
data = gen_df[dt_start:dt_end][var]


max_embedding_dimension = 10
max_time_delay = 10
stride = 1
um  = UMAP(random_state=0, n_components=4)
se = SpectralEmbedding(n_components=4, random_state=0)
y = data
var_name = var
persistence = VietorisRipsPersistence(
    homology_dimensions=[0, 1, 2],
    n_jobs=-1
)

periodicity_analysis(max_embedding_dimension=max_embedding_dimension,
                     max_time_delay=max_time_delay,
                     stride=stride,  
                     y=y,
                     var_name=var_name,
                     cluster=um,
                     persistence=persistence)

### Anomalías Nino 3

In [ ]:
var = 'anom_nino3'
dt_start = datetime(1980, 1, 1)
dt_end = datetime(1995, 1, 1)
data = gen_df[dt_start:dt_end][var]

#UMAP
max_embedding_dimension = 10
max_time_delay = 10
stride = 1
um  = UMAP(random_state=0, n_components=4)
y = data
var_name = var
persistence = VietorisRipsPersistence(
    homology_dimensions=[0, 1, 2],
    n_jobs=-1
)

periodicity_analysis(max_embedding_dimension=max_embedding_dimension,
                     max_time_delay=max_time_delay,
                     stride=stride,
                     y=y,
                     var_name=var_name,
                     cluster=um,
                     persistence=persistence
                     )

### Nino 4

In [ ]:
# 1980, 1995
var = 'nino4'
dt_start = datetime(1980, 1, 1)
dt_end = datetime(1997, 1, 1)
data = gen_df[dt_start:dt_end][var]

max_embedding_dimension = 10
max_time_delay = 10
stride = 1
um  = UMAP(random_state=0, n_components=4)
y = data
var_name = var
persistence = VietorisRipsPersistence(
    homology_dimensions=[0, 1, 2],
    n_jobs=-1
)


periodicity_analysis(max_embedding_dimension=max_embedding_dimension,
                     max_time_delay=max_time_delay,
                     stride=stride,
                     y=y,
                     var_name=var_name,
                     cluster=um,
                     persistence=persistence)

### Anomalías Nino 4

In [ ]:
# 1980, 1995
var = 'anom_nino4'
data = gen_df[var]#[dt_start:dt_end][var]

max_embedding_dimension = 10
max_time_delay = 10
stride = 1
um  = UMAP(random_state=0, n_components=4)
y = data
var_name = var
persistence = VietorisRipsPersistence(
    homology_dimensions=[0, 1, 2],
    n_jobs=-1
)

periodicity_analysis(max_embedding_dimension=max_embedding_dimension,
                     max_time_delay=max_time_delay,
                     stride=stride,
                     y=y,
                     var_name=var_name,
                     cluster=um,
                     persistence=persistence)

### Nino 3.4

In [ ]:
# 1980, 1995
var = 'nino3.4'
dt_start = datetime(1982, 1, 1)
dt_end = datetime(2006, 1, 1)
data = gen_df[dt_start:dt_end][var]
max_embedding_dimension = 10
max_time_delay = 13
stride = 1
um  = UMAP(random_state=0, n_components=4)
y = data
var_name = var
persistence = VietorisRipsPersistence(
    homology_dimensions=[0, 1, 2],
    n_jobs=-1
)

periodicity_analysis(max_embedding_dimension=max_embedding_dimension,
                     max_time_delay=max_time_delay,
                     stride=stride,
                     y=y,
                     var_name=var_name,
                     cluster=um,
                     persistence=persistence)

### Anomalías Nino 3.4

In [ ]:
# 1980, 1995
var = 'anom_nino3.4'
delta = 15
start = 1980  - 6*delta
dt_start = datetime(1980, 1, 1)
dt_end = datetime(1995, 1, 1)
data = gen_df[var]#[dt_start:dt_end][var]
data = gen_df[dt_start:dt_end][var]

max_embedding_dimension = 10
max_time_delay = 10
stride = 1
um  = UMAP(random_state=0, n_components=4)
y = data
var_name = var
persistence = VietorisRipsPersistence(
    homology_dimensions=[0, 1, 2],
    n_jobs=-1
)

periodicity_analysis(max_embedding_dimension=max_embedding_dimension,
                     max_time_delay=max_time_delay,
                     stride=stride,
                     y=y,
                     var_name=var_name,
                     cluster=um,
                     persistence=persistence)

## Reconocimiento de patrones

In [ ]:
var = 'nino3.4'

dt_start = datetime(1980, 1, 1)
dt_end = datetime(2023, 1, 1)
data = gen_df[dt_start:dt_end][['year', 'month', f'{var}', f'anom_{var}']]

In [ ]:
px.histogram(data, x=f'{var}')

### Moño

Se proyecta el año, mes, temperatura e índice de anomalía de Nino3.4 a el índice de anomalía Nino3.4

In [ ]:
var = 'nino3.4'

dt_start = datetime(1980, 1, 1)
dt_end = datetime(2023, 1, 1)
data = gen_df[dt_start:dt_end][['year', 'month', f'{var}', f'anom_{var}']]

mapper = km.KeplerMapper(verbose=0)
projected_data = mapper.fit_transform(
    X=data.to_numpy(), 
    projection=[
        3
    ]
)
covering=km.Cover(n_cubes=4,
                  perc_overlap=0.2)

G = mapper.map(projected_data, data,
               clusterer=KMeans(n_clusters=4, random_state=0, n_init='auto'),
               cover=covering)
fig = mapper.visualize(G, 
                path_html='graphs/noaa_temp_anom_nino34.html',
                title=f'Dates, Nino 3.4 temps and Anom Nino 3.4 Idx proj. to Nino 3.4 Anom Idx',
                custom_tooltips = data.index,
                color_values = gen_df[dt_start:dt_end]['nino3.4'],
                color_function_name = 'Nino 3.4 temperatures',
                node_color_function=np.array(['max', 'average','std','sum', 'min', 'median'])
            )

!explorer.exe "graphs\\noaa_temp_anom_nino34.html"

### Cuerda

Se genera una clusterización basada solamente en las mediciones de anomalías de Nino3.4 (forma un gusanito)

In [ ]:
# Previous 1870 - 2018

dt_start = datetime(1870, 1, 1)
dt_end = datetime(2018, 1, 1)
data = df[dt_start:dt_end]
data['month'] = data.index.month
data['year'] = data.index.year
data = data[['month', 'year', 'anom_nino3.4']]

mapper = km.KeplerMapper(verbose=0)

projected_data = mapper.fit_transform(
    X=data.to_numpy(), 
    projection=[
        2
    ]
)
covering=km.Cover(n_cubes=2,
                  perc_overlap=0.1)

G = mapper.map(projected_data, data,
               clusterer=KMeans(n_clusters=4, 
                                random_state=0, 
                                n_init='auto'),
               cover=covering)
fig = mapper.visualize(G,
                path_html='graphs/climateai_anom_nino34.html',
                title=f'Dates and Nino 3.4 index proj. to Nino 3.4 Index',
                custom_tooltips = data.index,
                color_values = data['anom_nino3.4'],
                color_function_name = 'Anom Nino 3.4',
                node_color_function=['max', 'average','std','sum','min']
            )

!explorer.exe "graphs\\climateai_anom_nino34.html"

In [ ]:
hot_node = data.iloc[G['nodes']['cube1_cluster3']]
cold_node = data.iloc[G['nodes']['cube0_cluster1']]
hot_node_2 = data.iloc[G['nodes']['cube1_cluster1']]
cold_node_2 = data.iloc[G['nodes']['cube0_cluster0']]

plt.figure(figsize=(10, 6))
# Plot the time series of anom_nino3.4 from the hot node
plt.plot(hot_node.index, 
         hot_node['anom_nino3.4'], 
         label='Hot node', 
         marker='o',
         markersize=3,
         lw=1)
# Plot the time series of anom_nino3.4 from the cold node
plt.plot(cold_node.index, 
         cold_node['anom_nino3.4'], 
         label='Cold node',
         marker='o',
         markersize=3,
         lw=1)
# Plot the time series of anom_nino3.4 from the hot node
plt.plot(hot_node_2.index,
            hot_node_2['anom_nino3.4'],
            label='Hot node 2',
            marker='o',
            markersize=3,
            lw=1)
# Plot the time series of anom_nino3.4 from the cold node
plt.plot(cold_node_2.index,
            cold_node_2['anom_nino3.4'],
            label='Cold node 2',
            marker='o',
            markersize=3,
            lw=1)

plt.plot
plt.xlabel('Date')
plt.ylabel('Anom Nino 3.4')
plt.title('Time series of anom_nino3.4 from the hot and cold nodes')
plt.legend()
plt.grid()
plt.show()

In [ ]:
dt_start = datetime(1870, 1, 1)
dt_end = datetime(2018, 1, 1)
data = df[dt_start:dt_end]
data['month'] = data.index.month
data['year'] = data.index.year
data = data[['month', 'year', 'anom_nino3.4']]

mapper = km.KeplerMapper(verbose=0)
projected_data = mapper.fit_transform(
    X=data.to_numpy(), 
    projection=[
        2
    ]
)

covering=km.Cover(n_cubes=3,
                  perc_overlap=0.2)

G = mapper.map(projected_data, data,
               clusterer=KMeans(n_clusters=4, 
                                random_state=0, 
                                n_init='auto'),
               cover=covering)
fig = mapper.visualize(G,
                path_html='graphs/climateai_anom_nino34_realistic.html',
                title=f'Dates, Nino 3.4 temps and Nino 3.4 index proj. to Nino 3.4 Index',
                custom_tooltips = data.index,
                color_values = data['anom_nino3.4'],
                color_function_name = 'Anom Nino 3.4',
                node_color_function=np.array(['average','std','sum','max','min'])
            )

!explorer.exe "graphs\\climateai_anom_nino34_realistic.html"

## Modelo de pronóstico

In [ ]:
from dateutil.relativedelta import relativedelta
from gtda.diagrams import Amplitude
from gtda.homology import VietorisRipsPersistence, WeakAlphaPersistence
from gtda.time_series import (
    Labeller,
    SlidingWindow,
    TakensEmbedding
)
from gtda.pipeline import make_pipeline
from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.ensemble import RandomForestRegressor
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError
from xgboost import XGBRegressor


In [ ]:
n_pred = 40
y = data['anom_nino3.4']

y_train, y_test = temporal_train_test_split(y, test_size=n_pred)
mape = MeanAbsolutePercentageError(symmetric=False)
smape = MeanAbsolutePercentageError(symmetric=True)

Lab = Labeller(size=1, func=np.max)
_, _ = Lab.fit_transform_resample(y_train, y_train)
SW = SlidingWindow(size=n_pred)
TE = TakensEmbedding(time_delay=1, dimension=5)
VR = VietorisRipsPersistence()
WA = WeakAlphaPersistence()
Ampl = Amplitude()
XGB = XGBRegressor()

pipe = make_pipeline(Lab, SW, TE, VR, Ampl, XGB)
pipe.fit(y_train, y_train)
y_pred = pipe.predict(y_train)

pipe

In [ ]:
print(f'{100*pipe.score(y_train, y_train) = :.2f} %')

In [ ]:
full_pred = pd.Series(
    y_pred,
    index=pd.date_range(start=y_train.index[n_pred], 
                        end=y_train.index[-1].to_pydatetime(),
                        freq='MS')
)
yt_plot = y_train[n_pred:] 
min_date, max_date = 1880, 1940
plt.figure(figsize=(10, 6))
plt.plot(yt_plot[datetime(min_date, 1, 1):datetime(max_date, 1, 1)], label='correct')
plt.plot(full_pred[datetime(min_date, 1, 1):datetime(max_date, 1, 1)], label='predicted')
plt.grid()
plt.legend()
plt.title(f'ML TDA informed Forecasting Model for Nino 3.4 Index {min_date}-{max_date}')
plt.xlabel('Date')
plt.ylabel('Anom Nino 3.4 Idx')
plt.savefig(f'graphs/forecasting_{min_date}_{max_date}.png',
            dpi=300,
            bbox_inches='tight')
plt.show()

In [ ]:
start_idx = y_train.index[-1].to_pydatetime() + relativedelta(months=1)
pred_idx = pd.date_range(start_idx, periods=n_pred, freq='MS')
pred_series = pd.Series(y_pred[-n_pred:], index=pred_idx)

plt.figure(figsize=(9, 4))
plt.plot(y_test, label='correct')
plt.plot(pred_series, label='predicted')
plt.legend()
plt.grid()
plt.title('ML TDA informed Forecasting Model')
plt.xlabel('Date')
plt.ylabel('Anom Nino 3.4 Idx')
plt.savefig(f'graphs/forecasting_test.png',
            dpi=300,
            bbox_inches='tight')
plt.show()

In [ ]:
# Correct index type to avoid error
pred_series.index = y_test.index

mape_error = mape(y_test, pred_series)
smape_error = smape(y_test, pred_series)

print(f'''Forecasting Summary:
{mape_error = :.2f} %
{smape_error = :.2f} %
''')
